In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import operator
import gc

In [2]:
PATH = '../data/'

PROP_DATA_PATH = PATH + 'Property dataset/'
TRN_DATA_PATH = PATH + 'training dataset/'
TEST_DATA_PATH = PATH + 'Test dataset/'

In [3]:
##### Accounts Profiles

accounts_table = pd.read_csv(TRN_DATA_PATH+'Accounts.csv')

accounts_table.head()

accounts_table.describe()

accts_table_cnts_features = [
                                'active_deals',
                                'activity_count',
                                'num_deals_as_client',
                                'num_deals_as_investor',
                                'number_of_properties',
                                'number_of_related_deals',
                                'number_of_related_properties',
                                'number_of_won_deals_as_client'
                            ]

accts_table_bool_vars = [
                            'buyer_book',
                            'servicing_contract',
                            'cmbs',
                            'consultant',
                            'correspondent',
                            'foreign',
                            'master_servicer',
                            'lender_book',
                            'loan_sales_book',
                            'loan_servicing'
                        ]

for var in accts_table_bool_vars:
    print(accounts_table[var].value_counts())

for var in accts_table_bool_vars:
    accounts_table[var].replace([True,False],[1,0],inplace=True)

accounts_table.head()

accts_ids = accounts_table.id_accs.values.tolist()

accts_profiles = accounts_table[accts_table_cnts_features+accts_table_bool_vars].values.astype('float')

True     2189
False     567
Name: buyer_book, dtype: int64
False    2700
True       56
Name: servicing_contract, dtype: int64
False    2749
True        7
Name: cmbs, dtype: int64
False    2756
Name: consultant, dtype: int64
False    2741
True       15
Name: correspondent, dtype: int64
False    2578
True      178
Name: foreign, dtype: int64
False    2754
True        2
Name: master_servicer, dtype: int64
False    1509
True     1247
Name: lender_book, dtype: int64
True     1792
False     964
Name: loan_sales_book, dtype: int64
False    2755
True        1
Name: loan_servicing, dtype: int64


In [63]:
##### Properties profiles
property_data = pd.read_csv(PROP_DATA_PATH+'Properties.csv')
property_data.drop('id_deals',axis=1,inplace=True)
display(property_data.head())
property_data_cnt_feats = [
                            'price_per_sq_ft',
                            'num_buildings',
                            'num_floors',
                            'num_parking_spaces',
                            'sale_amount__c',
                            'size_sf',
                            'year_built',
                            'city',
                            'county',
                            'market',
                            'class',
                            'property_type_1',
#                             'sale_status',
#                             'portfolio',

                        ]

property_ids = property_data.id_props.values.tolist()

property_profiles = property_data[property_data_cnt_feats].fillna(0).values

def get_property_feature_vector(idx):
    property_feat_vec = property_profiles[idx]
    return property_feat_vec

def get_similar_accounts(acct_id):
    acct_id_idx = accts_ids.index(acct_id)
    acct_id_feature_vector = accts_profiles[acct_id_idx].astype('float')
    reccs = [(i,scipy.spatial.distance.cosine(acct_id_feature_vector,apf.astype('float')))\
             for i,apf in enumerate(accts_profiles)]
    reccs.sort(key=operator.itemgetter(1))
    return reccs

# array([2.22280e+02, 1.00000e+00, 2.00000e+00, 4.00000e+02, 2.25000e+07,
#        1.01223e+05, 2.00100e+03])

# array([4.29969e+03, 0.00000e+00, 0.00000e+00, 0.00000e+00, 8.00000e+07,
#        1.86060e+04, 2.00300e+03])

,id_props,building_status,building_tax_expenses,city,class,county,sale_status,portfolio,market,num_buildings,...,price_per_sq_ft,price_per_unit,property_type_1,region__c,sale_amount__c,sale_date__c,size_acres,size_sf,size_units,year_built
0,a0I2A00000XQGKlUAP,Existing,2009 Tax @ $1.25/sf,City-0488,Class B,County-0172,N,False,Market-0059,1.0,...,46.59,18350000.00,Industrial,Southwest,1.835000e+07,7/10/2015,3.760,393862.0,1.0,1980.0
1,a0I2A00000XQGKnUAP,NaN,NaN,City-1975,Class A,County-0465,Y,True,Market-0239,NaN,...,1141.55,2109090.91,Multi-Housing,Southeast,2.144058e+07,12/31/2016,19.600,508080.0,275.0,2014.0
2,a0I2A00000XQGKoUAP,Existing,"2016 Tax @ $5.63/sf; 2007 Ops @ $19.58/sf, 201...",City-1415,Class B,County-0452,N,False,Market-0165,1.0,...,1506.65,NaN,Retail,Northeast,2.754156e+07,9/30/2016,0.266,18280.0,NaN,1902.0
3,a0I2A00000XQGKpUAP,NaN,NaN,City-1415,Class A,County-0452,N,False,Market-0165,1.0,...,580.83,1220689.66,NaN,Northeast,1.416000e+08,1/21/2016,0.530,243789.0,116.0,NaN
4,a0I2A00000XQGKqUAP,NaN,NaN,City-0941,NaN,County-0284,N,False,Market-0106,NaN,...,NaN,NaN,Self-Storage,Southwest,0.000000e+00,9/30/2014,NaN,NaN,NaN,NaN


In [72]:
property_data.isnull().sum()

id_props                     0
building_status           8217
building_tax_expenses     8743
city                         0
class                     6948
county                       0
sale_status                193
portfolio                    0
market                       1
num_buildings             7550
num_floors                4672
num_parking_spaces       10468
occupancy_rate            4052
price_per_sq_ft           6437
price_per_unit           12621
property_type_1            311
region__c                    5
sale_amount__c            6440
sale_date__c              6093
size_acres                4354
size_sf                   1568
size_units               10059
year_built                1858
dtype: int64

In [134]:
# property_data.append(a,sort=False)

pd.concat([property_data, a], axis=1, sort=False)

,id_props,building_status,building_tax_expenses,city,class,county,sale_status,portfolio,market,num_buildings,...,building_type_Hotel-Lodging,building_type_Industrial,building_type_Land,building_type_Medical Office Building,building_type_Multi-Housing,building_type_Office,building_type_Other,building_type_Retail,building_type_Self-Storage,building_type_Seniors Housing
0,a0I2A00000XQGKlUAP,Existing,2009 Tax @ $1.25/sf,City-0488,Class B,County-0172,N,False,Market-0059,7.816017,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,a0I2A00000XQGKnUAP,NaN,NaN,City-1975,Class A,County-0465,Y,True,Market-0239,44.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,a0I2A00000XQGKoUAP,Existing,"2016 Tax @ $5.63/sf; 2007 Ops @ $19.58/sf, 201...",City-1415,Class B,County-0452,N,False,Market-0165,1.123153,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,a0I2A00000XQGKpUAP,NaN,NaN,City-1415,Class A,County-0452,N,False,Market-0165,1.123153,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a0I2A00000XQGKqUAP,NaN,NaN,City-0941,NaN,County-0284,N,False,Market-0106,6.071429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,a0I2A00000XQGKrUAP,Existing,2016 Tax @ $5.42/sf,City-0210,Class A,County-0613,N,False,Market-0032,1.273504,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,a0I2A00000XQGKsUAP,Existing,2015 Tax @ $1158.52/Unit,City-0649,Class B,County-0624,N,False,Market-0059,16.454545,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,a0I2A00000XQGKtUAP,NaN,NaN,City-0648,Class A,County-0361,N,False,Market-0076,17.600000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,a0I2A00000XQGKvUAP,NaN,NaN,City-0246,NaN,County-0501,N,False,Market-0032,1.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,a0I2A00000XQGKwUAP,NaN,NaN,City-0424,NaN,County-0069,N,False,Market-0034,8.916667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [135]:
property_data.columns

Index(['id_props', 'building_status', 'building_tax_expenses', 'city', 'class',
       'county', 'sale_status', 'portfolio', 'market', 'num_buildings',
       'num_floors', 'num_parking_spaces', 'occupancy_rate', 'price_per_sq_ft',
       'price_per_unit', 'property_type_1', 'region__c', 'sale_amount__c',
       'sale_date__c', 'size_acres', 'size_sf', 'size_units', 'year_built'],
      dtype='object')

In [128]:
property_data.shape

(18298, 23)

In [123]:
a = pd.get_dummies(property_data['property_type_1'],prefix='building_type',dtype='float')

In [75]:
features_to_impute = [
                        'price_per_sq_ft',
                        'num_buildings',
                        'num_floors',
                        'num_parking_spaces',
                        'sale_amount__c',
                        'size_sf',
                        'year_built'
                     ]

In [94]:
for feat in features_to_impute:
    temp = property_data.groupby(['city'])[feat].agg('mean').reset_index()
    cities = temp['city'].values.tolist()
    feat_means = temp[feat].values.tolist()
    for ct in cities:
        property_data.loc[property_data['city']==ct,feat] = temp.loc[temp['city']==ct][feat].values[0]

In [95]:
property_data.isnull().sum()

id_props                     0
building_status           8217
building_tax_expenses     8743
city                         0
class                     6948
county                       0
sale_status                193
portfolio                    0
market                       1
num_buildings             1039
num_floors                 724
num_parking_spaces        4272
occupancy_rate            4052
price_per_sq_ft            795
price_per_unit           12621
property_type_1            311
region__c                    5
sale_amount__c             802
sale_date__c              6093
size_acres                4354
size_sf                    158
size_units               10059
year_built                 249
dtype: int64

In [64]:
property_profiles.shape

(18298, 12)

In [54]:
property_data.columns

Index(['id_props', 'building_status', 'building_tax_expenses', 'city', 'class',
       'county', 'sale_status', 'portfolio', 'market', 'num_buildings',
       'num_floors', 'num_parking_spaces', 'occupancy_rate', 'price_per_sq_ft',
       'price_per_unit', 'property_type_1', 'region__c', 'sale_amount__c',
       'sale_date__c', 'size_acres', 'size_sf', 'size_units', 'year_built'],
      dtype='object')

In [118]:
cols = [
            'building_status',
            'class',
            'sale_status',
            'portfolio',
            'property_type_1'
       ]

In [121]:
for col in cols:
    display(property_data[col].value_counts())

Existing              9728
Under Construction     139
Proposed               108
Demolished             106
Name: building_status, dtype: int64

Class A    5430
Class B    4882
Class C    1038
Name: class, dtype: int64

N    16501
Y     1604
Name: sale_status, dtype: int64

False    11926
True      6372
Name: portfolio, dtype: int64

Multi-Housing              5627
Office                     3744
Retail                     3474
Industrial                 2492
Hotel-Lodging               896
Medical Office Building     451
Self-Storage                404
Land                        351
Seniors Housing             276
Other                       220
Biotech                      52
Name: property_type_1, dtype: int64

In [137]:
for i,col in enumerate(cols):
#     display(d1[col].value_counts())
    a = pd.get_dummies(property_data[col],prefix=col+'_',dtype='float')
    if i == 0:
        ttt = pd.concat([property_data, a], axis=1, sort=False)
    else:
        ttt = pd.concat([ttt, a], axis=1, sort=False)

In [141]:
ttt.columns

Index(['id_props', 'building_status', 'building_tax_expenses', 'city', 'class',
       'county', 'sale_status', 'portfolio', 'market', 'num_buildings',
       'num_floors', 'num_parking_spaces', 'occupancy_rate', 'price_per_sq_ft',
       'price_per_unit', 'property_type_1', 'region__c', 'sale_amount__c',
       'sale_date__c', 'size_acres', 'size_sf', 'size_units', 'year_built',
       'building_status__Demolished', 'building_status__Existing',
       'building_status__Proposed', 'building_status__Under Construction',
       'class__Class A', 'class__Class B', 'class__Class C', 'sale_status__N',
       'sale_status__Y', 'portfolio__False', 'portfolio__True',
       'property_type_1__Biotech', 'property_type_1__Hotel-Lodging',
       'property_type_1__Industrial', 'property_type_1__Land',
       'property_type_1__Medical Office Building',
       'property_type_1__Multi-Housing', 'property_type_1__Office',
       'property_type_1__Other', 'property_type_1__Retail',
       'property_type

In [114]:
d1.property_type_1.value_counts()

Multi-Housing              6616
Office                     4887
Retail                     3798
Industrial                 2644
Hotel-Lodging               986
Medical Office Building     467
Self-Storage                394
Seniors Housing             373
Land                        283
Other                       165
Biotech                      58
Name: property_type_1, dtype: int64

In [55]:
property_data['property_type_1'].value_counts()

Multi-Housing              5627
Office                     3744
Retail                     3474
Industrial                 2492
Hotel-Lodging               896
Medical Office Building     451
Self-Storage                404
Land                        351
Seniors Housing             276
Other                       220
Biotech                      52
Name: property_type_1, dtype: int64

In [7]:
# opportunities_table = pd.read_csv(TRN_DATA_PATH+'Opportunities.csv')

# deal_to_properties_table = pd.read_csv(TRN_DATA_PATH+'Deals_to_Properties.csv')

In [17]:
# accounts_to_properties_table = pd.read_csv(TRN_DATA_PATH+'Accounts_properties.csv')
# accounts_to_properties_table.drop('id_deals',axis=1,inplace=True)

temp = pd.merge(accounts_to_properties_table,property_data,on=['id_props'],how='left')

In [32]:
temp[['id_accs','city','county']].groupby(['id_accs','city'])['county'].agg('count').reset_index()

,id_accs,city,county
0,0012A000023XlCCQA0,City-1406,1
1,0012A000023XlNjQAK,City-0020,1
2,0012A000023XlNjQAK,City-0045,1
3,0012A000023XlNjQAK,City-0070,1
4,0012A000023XlNjQAK,City-0090,1
5,0012A000023XlNjQAK,City-0386,1
6,0012A000023XlNjQAK,City-0435,1
7,0012A000023XlNjQAK,City-0745,1
8,0012A000023XlNjQAK,City-0936,2
9,0012A000023XlNjQAK,City-1013,1


In [61]:
##### Accounts profiling on the basis of Property Features

accounts_to_properties_table = pd.read_csv(TRN_DATA_PATH+'Accounts_properties.csv')
accounts_to_properties_table.drop('id_deals',axis=1,inplace=True)

d1 = pd.merge(accounts_to_properties_table,property_data,on=['id_props'],how='left')

# Define all the groupby transformations
GROUPBY_AGGREGATIONS = [
        {'groupby': ['id_accs'], 'select': 'price_per_sq_ft', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'num_buildings', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'num_floors', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'num_parking_spaces', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'sale_amount__c', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'size_sf', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'year_built', 'agg': 'mean'},
#         {'groupby': ['id_accs'], 'select': 'county', 'agg': 'nunique'},
#         {'groupby': ['id_accs'], 'select': 'city', 'agg': 'nunique'},
#         {'groupby': ['id_accs'], 'select': 'market', 'agg': 'nunique'},
        {'groupby': ['id_accs','city'], 'select': 'county', 'agg': 'count'},
        {'groupby': ['id_accs','county'], 'select': 'city', 'agg': 'count'},
        {'groupby': ['id_accs','market'], 'select': 'county', 'agg': 'count'},
#         {'groupby': ['id_accs'], 'select': 'portfolio', 'agg': 'count'},
#         {'groupby': ['id_accs'], 'select': 'sale_status', 'agg': 'count'},
        {'groupby': ['id_accs','class'], 'select': 'county', 'agg': 'count'},
        {'groupby': ['id_accs','property_type_1'], 'select': 'county', 'agg': 'count'}
    ]

# Apply all the groupby transformations
for spec in GROUPBY_AGGREGATIONS:
    
    # Name of the aggregation we're applying
    agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']
    
    # Name of new feature
    new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])
    # print(new_feature)
    # Info
    print("Grouping by {}, and aggregating {} with {}".format(
        spec['groupby'], spec['select'], agg_name
    ))
    
    # Unique list of features to select
    all_features = list(set(spec['groupby'] + [spec['select']]))
    # print(all_features)
    # Perform the groupby
    print(new_feature)
    gp = d1[all_features]. \
        groupby(spec['groupby'])[spec['select']]. \
        agg(spec['agg']). \
        reset_index(). \
        rename(index=str, columns={spec['select']: new_feature})
#     display(gp.head())
    d1 = d1.merge(gp, on=spec['groupby'], how='left')
    # Clear memory
    del gp
    gc.collect()

Grouping by ['id_accs'], and aggregating price_per_sq_ft with mean
id_accs_mean_price_per_sq_ft
Grouping by ['id_accs'], and aggregating num_buildings with mean
id_accs_mean_num_buildings
Grouping by ['id_accs'], and aggregating num_floors with mean
id_accs_mean_num_floors
Grouping by ['id_accs'], and aggregating num_parking_spaces with mean
id_accs_mean_num_parking_spaces
Grouping by ['id_accs'], and aggregating sale_amount__c with mean
id_accs_mean_sale_amount__c
Grouping by ['id_accs'], and aggregating size_sf with mean
id_accs_mean_size_sf
Grouping by ['id_accs'], and aggregating year_built with mean
id_accs_mean_year_built
Grouping by ['id_accs', 'city'], and aggregating county with count
id_accs_city_count_county
Grouping by ['id_accs', 'county'], and aggregating city with count
id_accs_county_count_city
Grouping by ['id_accs', 'market'], and aggregating county with count
id_accs_market_count_county
Grouping by ['id_accs', 'class'], and aggregating county with count
id_accs_class

In [99]:
d1.head()

,id,id_accs,id_props,building_status,building_tax_expenses,city,class,county,sale_status,portfolio,...,id_accs_mean_num_floors,id_accs_mean_num_parking_spaces,id_accs_mean_sale_amount__c,id_accs_mean_size_sf,id_accs_mean_year_built,id_accs_city_count_county,id_accs_county_count_city,id_accs_market_count_county,id_accs_class_count_county,id_accs_property_type_1_count_county
0,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,NaN,NaN,City-1406,NaN,County-0472,N,False,...,NaN,NaN,62500000.0,1.0,1907.0,1.0,1.0,1.0,NaN,1.0
1,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,Existing,2012 Tax @ $2.78/sf; 2007 Combined Est Tax/Ops...,City-1627,NaN,County-0154,N,False,...,7.5,303.333333,46980000.0,120597.2,1942.6,1.0,1.0,2.0,NaN,5.0
2,a062A00001VmQByQAN,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.5,303.333333,46980000.0,120597.2,1942.6,NaN,NaN,NaN,NaN,NaN
3,a062A00001VmQC2QAN,0012A000023XmOxQAK,a0I2A00000XwZyBUAV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.5,303.333333,46980000.0,120597.2,1942.6,NaN,NaN,NaN,NaN,NaN
4,a062A00001VmW8wQAF,0012A000023XmOxQAK,a0I2A00000ZGZClUAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.5,303.333333,46980000.0,120597.2,1942.6,NaN,NaN,NaN,NaN,NaN


In [108]:
d1.shape

(70350, 37)

In [112]:
property_data.sale_status.value_counts()

N    16501
Y     1604
Name: sale_status, dtype: int64

In [111]:
d1.sale_status.value_counts()

N    19526
Y     1279
Name: sale_status, dtype: int64

In [107]:
d1.groupby(['id_accs'])['sale_status'].apply(lambda x: x.value_counts())

id_accs              
0012A000023XlCCQA0  N      1
0012A000023XlNjQAK  N     14
0012A000023XlQFQA0  Y      3
                    N      1
0012A000023XlSVQA0  N      1
0012A000023XlStQAK  N     21
                    Y      3
0012A000023XlUGQA0  N     10
0012A000023XlVSQA0  N    449
                    Y      2
0012A000023XlVXQA0  N     21
0012A000023XlVhQAK  N     30
0012A000023XlVkQAK  N    155
                    Y      1
0012A000023XlVrQAK  N      1
0012A000023XlVwQAK  N      7
0012A000023XlWBQA0  N      3
0012A000023XlWMQA0  N     18
0012A000023XlWvQAK  N      9
0012A000023XlX9QAK  N     24
0012A000023XlXsQAK  N      2
0012A000023XlYCQA0  N      1
0012A000023XlYMQA0  N     11
0012A000023XlYRQA0  N     73
0012A000023XlYgQAK  N     58
0012A000023XlYlQAK  N     25
0012A000023XlYqQAK  N      3
0012A000023XlYvQAK  N     44
0012A000023XlZ0QAK  N      3
0012A000023XlZ5QAK  N    225
                        ... 
0012A00002Bg1kGQAR  N     22
                    Y      1
0012A00002Bg1rfQAB  N

In [62]:
d11 = d1[
            [
                'id_accs',
                'id_accs_mean_price_per_sq_ft',
                'id_accs_mean_num_buildings',
                'id_accs_mean_num_floors',
                'id_accs_mean_num_parking_spaces',
                'id_accs_mean_sale_amount__c',
                'id_accs_mean_size_sf',
                'id_accs_mean_year_built',
                'id_accs_city_count_county',
                'id_accs_county_count_city',
                'id_accs_market_count_county',
                'id_accs_class_count_county',
                'id_accs_property_type_1_count_county'
            ]
        ]

d11_dash = d11.drop_duplicates()

acc_profiles_wrt_property_features = d11_dash.fillna(0).values[:,1:]

acc_profiles_wrt_property_features.shape

(14352, 12)

In [58]:
test_data = pd.read_csv(TEST_DATA_PATH+'Test_Data.csv')

test_data.head()

ids = list(test_data['id_accs'].values)

exclude = set(list(accounts_to_properties_table.id_props.unique()))

print(len(exclude))

63001


In [69]:
property_profiles[0]

array([46.59, 1.0, 1.0, 56.0, 18350000.0, 393862.0, 1980.0, 'City-0488',
       'County-0172', 'Market-0059', 'Class B', 'Industrial'],
      dtype=object)

In [68]:
acc_profiles_wrt_property_features[0]

array([62500000.0, 1.0, 0.0, 0.0, 62500000.0, 1.0, 1907.0, 1.0, 1.0, 1.0,
       0.0, 1.0], dtype=object)

In [65]:
def filter_similar_accts(tuples_list):
    uniq = {}
    x = np.array([each[1] for each in tuples_list])
    p = np.percentile(x,1)
    print("accounts_percentile",p)
    for tl in tuples_list:
        if tl[1] not in uniq:
            uniq[tl[1]] = tl[0]     
        if tl[1] > p:
            break
    return uniq

results = {}

for i,_id in enumerate(ids):
    most_similar_accts = get_similar_accounts(_id)
    print(len(most_similar_accts))
    filtered_most_similar_accts = filter_similar_accts(most_similar_accts)
    filtered_most_similar_accts = filtered_most_similar_accts.values()
    print(len(filtered_most_similar_accts))
    print(filtered_most_similar_accts)
    top_reccs = []
    for most_similar_acct in filtered_most_similar_accts:
#         if most_similar_acct > 2727:
#             continue
        v1 = None
        v1 = acc_profiles_wrt_property_features[most_similar_acct].astype('float')
        reccs = []
        reccs = [(i,scipy.spatial.distance.cosine(v1,pf))\
                 if not np.isnan(scipy.spatial.distance.cosine(v1,pf)) else (i,0)
                 for i,pf in enumerate(property_profiles)]
        reccs.sort(key=operator.itemgetter(1))
        reccs = [r for r in reccs if r[1]>0]
        
        x = np.array([each[1] for each in reccs])
        if x.size==0:
            continue
        p = np.percentile(x,1.5)
        top_reccs.extend([property_ids[r[0]] for r in reccs if r[1]<=p])
    results[_id] = list(set(top_reccs)-exclude)
    print(len(results[_id]))

2756
accounts_percentile 1.330248456610228e-07
29
dict_values([79, 367, 1061, 426, 221, 1911, 2125, 1297, 1705, 1102, 1438, 1278, 29, 176, 1615, 315, 1703, 451, 586, 1032, 1371, 1468, 547, 755, 127, 140, 2153, 1612, 797])


TypeError: can't multiply sequence by non-int of type 'float'

In [46]:
property_ids[2372]

'a0I2A00000XQHF3UAP'

In [48]:
property_profiles[18257]

array([0., 0., 0., 0., 0., 0., 0.])

In [47]:
property_profiles[2372]

array([0., 0., 0., 0., 0., 0., 0.])

In [63]:
import csv
with open('subm8.csv','w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['id_accs','id_prop'])
    for resid,res in results.items():
        for r in res:
            row = [resid]+[r]
            writer.writerow(row)